# Automated Guitar Amp Modelling Training on Google Colab

This notebook sets up and runs the neural network training for guitar amplifier/distortion pedal modelling using PyTorch.

## Setup Instructions:
1. Make sure you have GPU runtime enabled: Runtime → Change runtime type → Hardware accelerator → GPU
2. Upload your training data files to Colab (or use the provided generate_test_signals.py)
3. Run the cells in order

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("WARNING: No GPU detected. This training will be very slow on CPU!")

In [ ]:
# Install required dependencies
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install numpy scipy matplotlib pyyaml tqdm librosa tensorboard
!pip install pytest black mypy  # Optional development tools

In [ ]:
# Clone the repository
!git clone https://github.com/diogoguedes666/Automated-GuitarAmpModelling-3.git
%cd Automated-GuitarAmpModelling-3

# Alternative: If you want to upload your local files instead of cloning
# from google.colab import files
# uploaded = files.upload()  # Upload your project files

In [ ]:
# Create necessary directories
!mkdir -p Data/train Data/val Data/test Results
!mkdir -p Configs

In [ ]:
# Option 1: Generate test signals if you don't have real training data
# This creates synthetic audio data for testing the training pipeline

import numpy as np
import scipy.io.wavfile as wavfile
import os

def generate_test_audio(filename, duration=10, sample_rate=44100, freq=440):
    """Generate a simple test audio file"""
    t = np.linspace(0, duration, int(sample_rate * duration), False)
    # Create a mix of sine waves for more interesting content
    audio = 0.3 * np.sin(freq * 2 * np.pi * t)  # 440 Hz sine wave
    audio += 0.2 * np.sin(2 * freq * 2 * np.pi * t)  # 880 Hz harmonic
    audio += 0.1 * np.sin(3 * freq * 2 * np.pi * t)  # 1320 Hz harmonic

    # Normalize to prevent clipping
    audio = audio / np.max(np.abs(audio)) * 0.8

    # Convert to 16-bit PCM
    audio_int16 = (audio * 32767).astype(np.int16)

    wavfile.write(filename, sample_rate, audio_int16)
    print(f"Generated test audio: {filename}")

# Generate test data
generate_test_audio("Data/train/dls2-input.wav", duration=30)
generate_test_audio("Data/train/dls2-target.wav", duration=30)  # This would be your "amp output"

generate_test_audio("Data/val/dls2-input.wav", duration=10)
generate_test_audio("Data/val/dls2-target.wav", duration=10)

generate_test_audio("Data/test/dls2-input.wav", duration=5)
generate_test_audio("Data/test/dls2-target.wav", duration=5)

print("Test data generation complete!")
print("Note: For real amp modelling, replace these with actual guitar -> amp recordings")

In [ ]:
# Option 2: Upload your own training data
# Uncomment the lines below if you want to upload your own WAV files

# from google.colab import files
# print("Upload your training input WAV file (guitar signal)")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/train/dls2-input.wav")

# print("Upload your training target WAV file (amp output)")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/train/dls2-target.wav")

# print("Upload validation input WAV file")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/val/dls2-input.wav")

# print("Upload validation target WAV file")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/val/dls2-target.wav")

# print("Upload test input WAV file")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/test/dls2-input.wav")

# print("Upload test target WAV file")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     os.rename(filename, f"Data/test/dls2-target.wav")

In [ ]:
# Create RNN3.json config file
config_content = '''{
  "model": "SimpleRNN",
  "input_size": 1,
  "output_size": 1,
  "num_blocks": 2,
  "hidden_size": 32,
  "unit_type": "LSTM",
  "skip_con": true,
  "segment_length": 22050,
  "batch_size": 512,
  "epochs": 100,
  "learn_rate": 0.005,
  "validation_f": 5,
  "validation_p": 20,
  "loss_fcns": {"ESRPre": 0.9, "DC": 0.1},
  "pre_filt": "high_pass",
  "cuda": 1,
  "weight_decay": 0.000001,
  "gradient_clip": 1.0
}
'''

with open('Configs/RNN3.json', 'w') as f:
    f.write(config_content)

print("Config file created: Configs/RNN3.json")

In [ ]:
# Verify data files exist
!ls -la Data/train/
!ls -la Data/val/
!ls -la Data/test/

In [ ]:
# Run a quick test to make sure the CoreAudioML modules can be imported
import sys
sys.path.append('.')

try:
    import CoreAudioML.miscfuncs as miscfuncs
    import CoreAudioML.training as training
    import CoreAudioML.dataset as dataset
    import CoreAudioML.networks as networks
    print("✓ All CoreAudioML modules imported successfully")
except Exception as e:
    print(f"✗ Import error: {e}")
    print("Check that all files are in the correct locations")

In [ ]:
# Start training!
# This will run the main training script with the RNN3 configuration

# First, let's do a quick dry run to make sure everything works
print("Testing training setup with 1 epoch...")
!python dist_model_recnet.py --load_config RNN3 --epochs 1 --batch_size 64 --device dls2

# If the test works, you can run the full training:
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --device dls2

In [ ]:
# If you want to run the full training (this will take time), uncomment and run:
# !python dist_model_recnet.py --load_config RNN3 --epochs 200 --device dls2

# Or run with different parameters:
# !python dist_model_recnet.py --load_config RNN3 --epochs 100 --batch_size 1024 --hidden_size 64 --device dls2

In [ ]:
# Check training results
!ls -la Results/

# If training completed, you should see directories like 'dls2-RNN3' with model files
# You can download the results:
# from google.colab import files
# !zip -r results.zip Results/
# files.download('results.zip')

In [ ]:
# Monitor GPU usage during training (run this in a separate cell while training)
# !nvidia-smi --query-gpu=timestamp,name,utilization.gpu,utilization.memory,memory.used,memory.free --format=csv -l 5

## Tips for Better Training:

1. **GPU Memory**: If you get CUDA out of memory errors, reduce `batch_size` in the config
2. **Training Time**: Start with fewer epochs (50-100) to test, then increase
3. **Model Size**: Larger `hidden_size` (64, 128) gives better quality but slower training
4. **Data Quality**: Real guitar recordings give much better results than synthetic data
5. **Monitoring**: Use TensorBoard to monitor training progress

## Troubleshooting:

- **Import errors**: Make sure all files are uploaded/cloned correctly
- **CUDA errors**: Ensure GPU runtime is enabled
- **Memory errors**: Reduce batch size or use gradient accumulation
- **Data errors**: Check that WAV files are mono, 44.1kHz, and properly matched

## Next Steps:

1. Download trained models from the Results folder
2. Test inference with `proc_audio.py` on new audio
3. Experiment with different configurations (RNN1.json, RNN2.json)
4. Try different architectures or hyperparameters